In [65]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random 

# source: https://www.kaggle.com/danofer/usp-drug-classification/version/1
drugs = pd.read_csv("usp_drug_classification.csv")

drug_names = list(drugs["drug_example"].values)
drug_names = drug_names[:10]

df = pd.DataFrame()
dates = []
station_ids = []
medication_names = []
medication_costs = []
expiration_dates = []
number_of_pills_lost = []
medication_cost = {drug: random.randint(1, 7500) for drug in drug_names}
expiration_date = {drug: random.randint(3, 45) for drug in drug_names}

for i in range(10000):
    cur_date = datetime(2021, random.randint(2, 5), random.randint(1, 28))
    dates.append(cur_date)
    medication_name = random.choice(drug_names)
    medication_names.append(medication_name)
    medication_costs.append(medication_cost[medication_name])
    expiration_dates.append(cur_date + timedelta(days=expiration_date[medication_name]))
    station_ids.append(random.randint(11111, 99999))
    number_of_pills_lost.append(random.randint(120, 1000))
    
# the test criteria is ultimately units and $ of medication waste
# or "running out"
df["date"] = dates
df["station_id"] = station_ids
df["medication_name"] = medication_names 
df["expiration_date"] = expiration_dates
df["medication_cost"] = medication_costs
df["number_of_pills_lost"] = number_of_pills_lost
df["despense_quantity"] = 1

In [50]:
df.head()

,date,station_id,medication_name,expiration_date,medication_cost,number_of_pills_lost,despense_quantity
0,2021-05-26,99762,diclofenac diethylamine,2021-07-07,5104,130,1
1,2021-05-11,72604,diclofenac potassium,2021-06-07,6449,156,1
2,2021-05-01,36266,diclofenac sodium,2021-06-14,4389,763,1
3,2021-05-18,16145,diclofenac potassium,2021-06-14,6449,168,1
4,2021-05-30,64175,diclofenac sodium,2021-07-13,4389,535,1


medication model:
* index column
* frequency count of medication per day - groupby on medication_name

station model:
* index column
* frequency count of station per day - groupby on station_id

In [66]:
# feature engineering

dates = list(df["date"].unique())
dates.sort()
date_mapper = {date: index for index, date in enumerate(dates)}
df["date_index"] = df["date"].map(date_mapper)

medication_df = pd.DataFrame()
for (medication_name, date), tmp in df.groupby(["medication_name", "date"]):
    dollars_lost = tmp["number_of_pills_lost"] * tmp["medication_cost"]
    dollars_lost = dollars_lost.sum()
    medication_df = medication_df.append({
        "date": tmp["date"].iloc[0],
        "medication_name": tmp["medication_name"].iloc[0],
        "medication_cost": tmp["medication_cost"].iloc[0],
        "medication_total_cost": tmp["medication_cost"].sum(),
        "date_index": tmp["date_index"].iloc[0],
        "freq_per_day": int(tmp.shape[0]),
        "dollars_lost": dollars_lost
    }, ignore_index=True)

In [67]:
# generate lagged features
def generate_lagged_features(df, col, num_lags):
    for lag in range(1, num_lags):
        df[col + f"_lagged_by_{lag}_days"] = df[col].shift(lag)
    df.fillna(0, inplace=True)
    return df

In [68]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

models = {}
for medication_name, tmp in medication_df.groupby("medication_name"):
    X = tmp[["date_index", "freq_per_day"]]
    num_lags = 7
    features = generate_lagged_features(X, "freq_per_day", num_lags)
    labels = features["freq_per_day"]
    features.drop("freq_per_day", inplace=True, axis=1)
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
    ran_forest_reg = RandomForestRegressor()
    print(train_features.shape)
    print(train_labels.shape)
    ran_forest_reg.fit(train_features, train_labels)
    y_pred = ran_forest_reg.predict(test_features)
    print(medication_name, "mse:", mean_squared_error(test_labels, y_pred))
    models[medication_name] = ran_forest_reg

<ipython-input-67-53e9536b15b5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col + f"_lagged_by_{lag}_days"] = df[col].shift(lag)
<ipython-input-67-53e9536b15b5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col + f"_lagged_by_{lag}_days"] = df[col].shift(lag)


(84, 7)
(84,)
celecoxib mse: 7.842260714285715
(84, 7)
(84,)
diclofenac mse: 11.605307142857143
(84, 7)
(84,)
diclofenac calcium mse: 5.349757142857143
(84, 7)
(84,)


<ipython-input-67-53e9536b15b5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col + f"_lagged_by_{lag}_days"] = df[col].shift(lag)
<ipython-input-67-53e9536b15b5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col + f"_lagged_by_{lag}_days"] = df[col].shift(lag)


diclofenac diethylamine mse: 10.624914285714286
(84, 7)
(84,)
diclofenac hydroxyethylpyrrolidine mse: 5.805014285714285
(84, 7)
(84,)


<ipython-input-67-53e9536b15b5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col + f"_lagged_by_{lag}_days"] = df[col].shift(lag)
<ipython-input-67-53e9536b15b5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col + f"_lagged_by_{lag}_days"] = df[col].shift(lag)


diclofenac potassium mse: 5.982849999999999
(84, 7)
(84,)
diclofenac sodium mse: 16.034160714285715
(84, 7)
(84,)


<ipython-input-67-53e9536b15b5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col + f"_lagged_by_{lag}_days"] = df[col].shift(lag)
<ipython-input-67-53e9536b15b5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col + f"_lagged_by_{lag}_days"] = df[col].shift(lag)


diflunisal mse: 9.934889285714288
(84, 7)
(84,)
etodolac mse: 14.886464285714288
(84, 7)
(84,)


<ipython-input-67-53e9536b15b5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col + f"_lagged_by_{lag}_days"] = df[col].shift(lag)
<ipython-input-67-53e9536b15b5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col + f"_lagged_by_{lag}_days"] = df[col].shift(lag)


fenoprofen mse: 12.927264285714285


In [70]:
features.head(50)

,date_index,freq_per_day_lagged_by_1_days,freq_per_day_lagged_by_2_days,freq_per_day_lagged_by_3_days,freq_per_day_lagged_by_4_days,freq_per_day_lagged_by_5_days,freq_per_day_lagged_by_6_days
1008,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1009,1.0,7.0,0.0,0.0,0.0,0.0,0.0
1010,2.0,10.0,7.0,0.0,0.0,0.0,0.0
1011,3.0,8.0,10.0,7.0,0.0,0.0,0.0
1012,4.0,4.0,8.0,10.0,7.0,0.0,0.0
1013,5.0,7.0,4.0,8.0,10.0,7.0,0.0
1014,6.0,5.0,7.0,4.0,8.0,10.0,7.0
1015,7.0,11.0,5.0,7.0,4.0,8.0,10.0
1016,8.0,9.0,11.0,5.0,7.0,4.0,8.0
1017,9.0,6.0,9.0,11.0,5.0,7.0,4.0
